<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/ai%2Fworking_on_count_vector/AI_StopWords_using_Stocknews_sentiment_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment.csv"), sep=",", header=None)

# Show DataFrame
df.show()

+--------+--------------------+
|     _c0|                 _c1|
+--------+--------------------+
|Positive|Blue Shield of Ca...|
|Positive|RSX: Russian Equi...|
|Positive|XLB: Hold Your Po...|
|Positive|Musgrave Minerals...|
|Positive|ViacomCBS to rest...|
|Positive|Oil climbs to one...|
|Negative|Australian regula...|
|Positive|The Wall Street J...|
|Positive|Ora Banda Mining ...|
|Positive|Google Pay Plans ...|
|Negative|Toyota, Honda Ang...|
| Neutral|ZY Investor Alert...|
| Neutral|Calima Energy spu...|
|Positive|These Marijuana S...|
|Positive|Moderna ETFs to R...|
| Neutral|MacroGenics Annou...|
| Neutral|Zymeworks Announc...|
| Neutral|BioXcel Therapeut...|
| Neutral|Merck to Present ...|
| Neutral|BeiGene to Presen...|
+--------+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
| Neutral|BeiGene to Presen...|    88|
+--------+--------------------+------+
only showing top 20 rows



In [7]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show()


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
| Neutral|BeiGene to Pre

In [8]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="Words")

In [9]:
# Transform DataFrame
reviewed = tokenizer.transform(df2)
reviewed.show()

+--------+--------------------+------+--------------------+
|   class|                text|length|               Words|
+--------+--------------------+------+--------------------+
|Positive|Blue Shield of Ca...|    80|[blue, shield, of...|
|Positive|RSX: Russian Equi...|    43|[rsx:, russian, e...|
|Positive|XLB: Hold Your Po...|    23|[xlb:, hold, your...|
|Positive|Musgrave Minerals...|   101|[musgrave, minera...|
|Positive|ViacomCBS to rest...|    49|[viacomcbs, to, r...|
|Positive|Oil climbs to one...|    51|[oil, climbs, to,...|
|Negative|Australian regula...|    67|[australian, regu...|
|Positive|The Wall Street J...|    90|[the, wall, stree...|
|Positive|Ora Banda Mining ...|    50|[ora, banda, mini...|
|Positive|Google Pay Plans ...|    43|[google, pay, pla...|
|Negative|Toyota, Honda Ang...|    69|[toyota,, honda, ...|
| Neutral|ZY Investor Alert...|    44|[zy, investor, al...|
| Neutral|Calima Energy spu...|    58|[calima, energy, ...|
|Positive|These Marijuana S...|    56|[t

In [10]:
# Remove stop words
stopremove = StopWordsRemover(inputCol="Words", outputCol="filtered")

In [11]:
# Transform new DataFrame
newFrame = stopremove.transform(reviewed)
newFrame.show()

+--------+--------------------+------+--------------------+--------------------+
|   class|                text|length|               Words|            filtered|
+--------+--------------------+------+--------------------+--------------------+
|Positive|Blue Shield of Ca...|    80|[blue, shield, of...|[blue, shield, ca...|
|Positive|RSX: Russian Equi...|    43|[rsx:, russian, e...|[rsx:, russian, e...|
|Positive|XLB: Hold Your Po...|    23|[xlb:, hold, your...|[xlb:, hold, posi...|
|Positive|Musgrave Minerals...|   101|[musgrave, minera...|[musgrave, minera...|
|Positive|ViacomCBS to rest...|    49|[viacomcbs, to, r...|[viacomcbs, restr...|
|Positive|Oil climbs to one...|    51|[oil, climbs, to,...|[oil, climbs, one...|
|Negative|Australian regula...|    67|[australian, regu...|[australian, regu...|
|Positive|The Wall Street J...|    90|[the, wall, stree...|[wall, street, jo...|
|Positive|Ora Banda Mining ...|    50|[ora, banda, mini...|[ora, banda, mini...|
|Positive|Google Pay Plans .

In [12]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
# tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="filtered", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [14]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [15]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df2)
cleaned = cleaner.transform(df2)

In [16]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[43204,50...|
|  1.0|(262145,[31536,12...|
|  1.0|(262145,[619,5765...|
|  1.0|(262145,[57093,58...|
|  1.0|(262145,[38640,57...|
|  1.0|(262145,[15862,73...|
|  2.0|(262145,[6993,206...|
|  1.0|(262145,[38308,63...|
|  1.0|(262145,[22459,41...|
|  1.0|(262145,[66208,84...|
|  2.0|(262145,[3091,636...|
|  0.0|(262145,[102692,1...|
|  0.0|(262145,[9129,233...|
|  1.0|(262145,[16004,89...|
|  1.0|(262145,[49185,12...|
|  0.0|(262145,[18731,22...|
|  0.0|(262145,[31793,45...|
|  0.0|(262145,[1097,216...|
|  0.0|(262145,[2690,281...|
|  0.0|(262145,[26190,51...|
+-----+--------------------+
only showing top 20 rows



In [17]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [18]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|               Words|            filtered|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Negative|"Apple rejects Ep...|    72|  2.0|["apple, rejects,...|["apple, rejects,...|(262144,[15625,38...|(262144,[15625,38...|(262145,[15625,38...|[-862.09154173162...|[1.0,8.7467280457...|       0.0|
|Negative|AGNC Investment (...|    75|  2.0|[agnc, investment...|[agnc, investment...|(262144,[50280,13...|(262144,[50280,13...|(262145,[50280,13...|[-503.71649253759...|[0.99999999999999.

In [19]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.617614
